In [12]:
import pandas as pd

pd.set_option('max_colwidth', 100)

# Gerenciador OS
import os, shutil

import glob

# Módulo gerenciador arquivos compactados
import zipfile

In [13]:
path_raw = '../../Data/Raw/'
path_processed = '../../Data/Processed/'

#Garantir diretório de processamento vazio
for filename in os.listdir(path_processed):
    file_path = os.path.join(path_processed, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Falha ao apagar %s. Razão: %s' % (file_path, e))

print("Extraindo arquivos xml...")
print()

# Iniciando a extração
zip_files = [f for f in glob.glob("".join([path_raw,"*.zip"]), recursive=False)]

for file_file in zip_files:
    with zipfile.ZipFile(file_file, 'r') as zip: 
        # Imprimir conteúdo do ZIP
        zip.printdir() 
        # Extraindo o ZIP 
        print('Extraindo arquivos...') 
        zip.extractall(path = path_processed)
print('Feito!')
print()

                 2020-08-26 23:42:34         1515
530_20200827_12889802.xml                      2020-08-26 23:42:36         1655
530_20200827_12889803.xml                      2020-08-26 23:42:40         2676
530_20200827_12889804.xml                      2020-08-26 23:43:36         2982
530_20200827_12889805.xml                      2020-08-26 23:43:14         1950
530_20200827_12889806.xml                      2020-08-26 23:42:36         1921
530_20200827_12889807.xml                      2020-08-26 23:47:02         1448
530_20200827_12889809.xml                      2020-08-26 23:46:58         1779
530_20200827_12889810.xml                      2020-08-26 23:46:18         1514
530_20200827_12889811.xml                      2020-08-26 23:46:58         2138
530_20200827_12889812.xml                      2020-08-26 23:42:16         1853
530_20200827_12889813.xml                      2020-08-26 23:37:16         2310
530_20200827_12889815.xml                      2020-08-26 23:47:18    

In [14]:
import bs4
import lxml
from datetime import datetime 

categoria_interesse = ['Atos do Congresso Nacional'
                      ,'Atos do Poder Executivo'
                      ,'Presidência da República'
                      ,'Ministério do Desenvolvimento Regional'
                      ,'Ministério de Minas e Energia'
                      ,'Ministério do Meio Ambiente'] 

conteudo = []
insumos_dou = []

# Todos os arquivos extraídos
xml_files = [f for f in glob.glob("".join([path_processed, "*.xml"]), recursive=False)]

# Processa cada arquivo
for file in xml_files:
        
    # Read the XML file
    with open("".join([path_processed, file]), "r", encoding='utf-8') as file:
        # Lê cada arquivo, retorna uma lista de linhas
        conteudo = file.readlines()
        
        # Combina linha em string
        conteudo = "".join(conteudo)

        # html.parser sendo utilizado, pois lxml retira informações CDATA
        bs_conteudo = bs4.BeautifulSoup(conteudo, "html.parser")

        #Separar artcategory de interesse
        bs_artcategory = bs_conteudo.find("article").get("artcategory")
           
        e_interesse = [categoria in bs_artcategory for categoria in categoria_interesse]
        
        # Verifica se categoria não é de interesse no texto
        if False in e_interesse:
            
            id = bs_conteudo.find("article").get("id")
            objeto = bs_conteudo.identifica.string if bs_conteudo.identifica.string else ''
            origem = bs_conteudo.find("article").get("pubname")
            data_publicacao = datetime.strptime(bs_conteudo.find("article").get("pubdate"), '%d/%m/%Y').strftime("%Y-%m-%d %H:%M:%S")
            informacao = bs_conteudo.texto.string if bs_conteudo.texto.string else ''
            data_inclusao = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

            # print('nome: ', bs_nome)
            # print('secao: ', bs_secao)
            # print('datap: ' + str(bs_data_publicacao))
            # print('categ: '+ bs_artcategory)
            # print('pag: '+ bs_num_pagina)
            # print('url: ' + bs_url_pdf)
            # print('idmateria: '+ bs_id_materia)
            # print('num_edicao: '+ bs_num_edicao)
            # print('texto: '+origem)
            # print('objeto: '+ objeto)
            # print('ementa: '+ informacao)
            
            insumo = (id, objeto, origem, data_publicacao, informacao, data_inclusao)
            
            insumos_dou.append(insumo)

In [15]:
# create DataFrame using data 
base_sem_interesse = pd.DataFrame(insumos_dou, columns =['id', 'objeto','origem','data_publicacao','informacao','data_inclusao']) 

In [16]:
base_sem_interesse.head(5)

,id,objeto,origem,data_publicacao,informacao,data_inclusao
0,15943207,AVISOs DE DISPENSAs DE LICITAÇÃO,DO3,2020-08-25 00:00:00,"<p class=""identifica"">AVISOs DE DISPENSAs DE LICITAÇÃO</p><p>ÓRGÃO: Prefeitura Municipal de Pres...",2020-09-12 16:41:30
1,16044513,PORTARIAS DE 24 DE AGOSTO DE 2020,DO2,2020-09-01 00:00:00,"<p class=""identifica"">PORTARIAS DE 24 DE AGOSTO DE 2020</p><p><strong>O DIRETOR-GERAL DA AGÊNCIA...",2020-09-12 16:41:30
2,16057413,EXTRATO DE CONTRATO Nº 26/2020 - UASG 150182,DO3,2020-09-01 00:00:00,"<p class=""identifica"">EXTRATO DE CONTRATO Nº 26/2020 - UASG 150182</p><p>Nº Processo: 2306915365...",2020-09-12 16:41:30
3,16004412,aviso de alteração Pregão Eletrônico nº 6/2020,DO3,2020-08-28 00:00:00,"<p class=""identifica"">aviso de alteração Pregão Eletrônico nº 6/2020</p><p>O município de Guaran...",2020-09-12 16:41:30
4,16062398,EXTRATO DE TERMO ADITIVO,DO3,2020-09-01 00:00:00,"<p class=""identifica"">EXTRATO DE TERMO ADITIVO</p><p>Processo TST nº 503.525/2019-4. Centro Clín...",2020-09-12 16:41:30


In [17]:
base_sem_interesse.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23649 entries, 0 to 23648
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               23649 non-null  object
 1   objeto           23649 non-null  object
 2   origem           23649 non-null  object
 3   data_publicacao  23649 non-null  object
 4   informacao       23649 non-null  object
 5   data_inclusao    23649 non-null  object
dtypes: object(6)
memory usage: 1.1+ MB


In [18]:
# Retira linhas vazias que não tenham a feature informação
base_sem_interesse.dropna(subset=['informacao'],inplace=True)

In [19]:
base_sem_interesse.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 23649 entries, 0 to 23648
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               23649 non-null  object
 1   objeto           23649 non-null  object
 2   origem           23649 non-null  object
 3   data_publicacao  23649 non-null  object
 4   informacao       23649 non-null  object
 5   data_inclusao    23649 non-null  object
dtypes: object(6)
memory usage: 1.3+ MB


In [20]:
# Realiza decode de html. Ex.: Troca &Ccedil; por Ç
import html

base_sem_interesse = base_sem_interesse.fillna("").applymap(html.unescape)

In [21]:
base_sem_interesse['informacao']

0        <p class="identifica">AVISOs DE DISPENSAs DE LICITAÇÃO</p><p>ÓRGÃO: Prefeitura Municipal de Pres...
1        <p class="identifica">PORTARIAS DE 24 DE AGOSTO DE 2020</p><p><strong>O DIRETOR-GERAL DA AGÊNCIA...
2        <p class="identifica">EXTRATO DE CONTRATO Nº 26/2020 - UASG 150182</p><p>Nº Processo: 2306915365...
3        <p class="identifica">aviso de alteração Pregão Eletrônico nº 6/2020</p><p>O município de Guaran...
4        <p class="identifica">EXTRATO DE TERMO ADITIVO</p><p>Processo TST nº 503.525/2019-4. Centro Clín...
                                                        ...                                                 
23644    <p class="identifica">PORTARIA Nº 19.755, DE 24 DE AGOSTO DE 2020</p><p>O COORDENADOR-GERAL DE B...
23645    <p class="identifica">AVISO DE LICITAÇÃO Nº 7003210032</p><p>Objeto: Suporte às atividades de pr...
23646    <p class="identifica">Aviso de Licitação Tomada de Preços nº 7/2020</p><p>Torna público Processo...
23647    <p class="

In [22]:
# Limpeza de tags HTML. Foi verificado que a partir de um certo momento os texto foram gravados com as tags HTML, provavelmente para facilitar a formatação no momento de visualização

from bleach.sanitizer import Cleaner

cleaner = Cleaner(tags=[], attributes={}, styles=[], protocols=[], strip=True, strip_comments=True, filters=None)

# Retira todas as tags html do texto do DOU
base_sem_interesse['informacao'] = base_sem_interesse['informacao'].apply(lambda x: cleaner.clean(x))

print("Retiradas tags HTML")

Retiradas tags HTML


In [51]:
base_sem_interesse.tail(5)

,id,objeto,origem,data_publicacao,informacao,data_inclusao
26944,16035432,AVISO DE LICITAÇÃO Nº 7003210032,DO3,2020-08-31 00:00:00,AVISO DE LICITAÇÃO Nº 7003210032Objeto: Suporte às atividades de projeto e gestão da produçãoAbe...,2020-09-06 18:49:03
26945,16090870,AVISO DE PRORROGAÇÃO PREGÃO ELETRÔNICO SRP Nº 10/2020,DO3,2020-09-03 00:00:00,AVISO DE PRORROGAÇÃO PREGÃO ELETRÔNICO SRP Nº 10/2020OBJETO: REGISTRO DE PREÇO PARA EVENTUAL AQU...,2020-09-06 18:49:03
26946,15982942,Aviso de Licitação Tomada de Preços nº 7/2020,DO3,2020-08-27 00:00:00,Aviso de Licitação Tomada de Preços nº 7/2020Torna público Processo nº 035/2020 - Tomada de Preç...,2020-09-06 18:49:03
26947,15931086,"EDITAL Nº 58, DE 21 DE AGOSTO DE 2020",DO3,2020-08-24 00:00:00,"EDITAL Nº 58, DE 21 DE AGOSTO DE 2020RETIFICAÇÃO DO EDITAL DE HOMOLOGAÇÃO DE RESULTADO FINAL Nº ...",2020-09-06 18:49:03
26948,16009580,DESPACHO DE 26 DE AGOSTO DE 2020,DO3,2020-08-28 00:00:00,DESPACHO DE 26 DE AGOSTO DE 2020Dispensa Eletrônica nº 029/2020.Objeto: aquisição de EPI. Assunt...,2020-09-06 18:49:03


In [23]:
base_sem_interesse.informacao

0        AVISOs DE DISPENSAs DE LICITAÇÃOÓRGÃO: Prefeitura Municipal de Presidente Prudente INTERESSADA: ...
1        PORTARIAS DE 24 DE AGOSTO DE 2020O DIRETOR-GERAL DA AGÊNCIA BRASILEIRA DE INTELIGÊNCIA DO GABINE...
2        EXTRATO DE CONTRATO Nº 26/2020 - UASG 150182Nº Processo: 23069153651202064.Regime de Execução: E...
3        aviso de alteração Pregão Eletrônico nº 6/2020O município de Guarantã do Norte/MT, representado ...
4        EXTRATO DE TERMO ADITIVOProcesso TST nº 503.525/2019-4. Centro Clínico Bandeirante Serviços e As...
                                                        ...                                                 
23644    PORTARIA Nº 19.755, DE 24 DE AGOSTO DE 2020O COORDENADOR-GERAL DE BENEFÍCIOS DO DEPARTAMENTO DE ...
23645    AVISO DE LICITAÇÃO Nº 7003210032Objeto: Suporte às atividades de projeto e gestão da produçãoAbe...
23646    Aviso de Licitação Tomada de Preços nº 7/2020Torna público Processo nº 035/2020 - Tomada de Preç...
23647    EDITAL Nº 

In [24]:
# Limpeza de ruído - Transformar em minúsculo, limpeza de caracteres especiais

import regex as re
import string

def limpa_ruido_texto(texto):
    texto = texto.lower()
    texto = re.sub('[%s]' % re.escape(string.punctuation), '', texto) # Retira pontuações
    texto = re.sub('\w*\d\w*', '', texto) # Retira números entre palavras
    texto = re.sub('§–', '', texto) # Casos específicos para os textos do DOU
    return texto

base_sem_interesse['informacao'] = base_sem_interesse['informacao'].apply(lambda x: limpa_ruido_texto(x))

base_sem_interesse['informacao'].head()

0    avisos de dispensas de licitaçãoórgão prefeitura municipal de presidente prudente interessada sa...
1    portarias de  de agosto de  diretorgeral da agência brasileira de inteligência do gabinete de se...
2    extrato de contrato nº   uasg  processo  de execução empreitada por preço unitáriordc eletrônico...
3    aviso de alteração pregão eletrônico nº  município de guarantã do nortemt representado pela pref...
4    extrato de termo aditivoprocesso tst nº  centro clínico bandeirante serviços e assistência médic...
Name: informacao, dtype: object

In [25]:
# Criação de matrix de termos utilizando CountVectorizer. 
# Estouro por falta de memória para a matrix de termos

import nltk

nltk.download('stopwords')

from sklearn.feature_extraction.text import CountVectorizer

nltk_pt_stopwords = nltk.corpus.stopwords.words('portuguese')

cv = CountVectorizer(stop_words=nltk_pt_stopwords, ngram_range = (1, 2), min_df = 0.01, max_df = 0.99)
cv_informacao = cv.fit_transform(base_sem_interesse['informacao'])
mt_informacao = pd.DataFrame(cv_informacao.toarray(), columns=cv.get_feature_names())
mt_informacao.index = base_sem_interesse['informacao'].index
mt_informacao

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/marcelo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,abaixo,abastecimento,aberto,abertura,abertura propostas,abril,acesso,acima,acolhimento,acompanhamento,...,água,área,áreas,âmbito,íntegra,órgão,órgãos,único,único art,úteis
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23644,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
23645,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23646,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23647,0,0,1,0,0,0,0,0,0,0,...,0,3,0,0,0,0,0,0,0,0


In [26]:
# Guardando a informação para uso futuro na EAD
import pickle

# Matriz de termos
mt_informacao.to_pickle('matrix_termos_base_SEM_interesse.pkl')

# Base de interesse higienizada
base_sem_interesse.to_pickle('base_SEM_interesse_limpa.pkl')
pickle.dump(cv_informacao, open('countvectorizer_base_SEM_interesse.pkl', 'wb'))
